In [1]:
#!rename -v 's/\.PNG/\.png/' *

In [2]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from keras import optimizers

In [3]:
DATADIR = './sample_Signature/genuine/'

In [4]:
list_os = os.listdir(DATADIR)

list_no = [str(i).zfill(2)  for i in range(1,31)] 

list_all = []

for i in range(1,31):
    a =[] 
    for j in range(1,6):
        str_a = 'NFI-0'+str(i).zfill(2)+'0'+str(j)+'0'+str(i).zfill(2)+'.png'
        a.append(DATADIR+str_a)
    
    list_all.append(a)

IMG_SIZE = 100


In [5]:
training_data = []

def create_training_data():
    for category,j in zip(list_all,list_no):  

        #path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = j # get the classification  (0 or a 1). 0=dog 1=cat
        
        for img in tqdm(category):  # iterate over each image per dogs and cats
            
            img_array = cv2.imread(img ,cv2.IMREAD_GRAYSCALE)  # convert to array
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
            training_data.append([new_array, class_num])  # add this to our training_data


create_training_data()
print(len(training_data))

100%|██████████| 5/5 [00:00<00:00, 145.79it/s]

150


In [6]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
    break

In [7]:
#print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

In [8]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

#print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = [int(i) for i in y]

In [9]:
from keras.utils import to_categorical

In [10]:
one_hot_labels = to_categorical(y)#, num_classes=30)
a = []
for i in one_hot_labels:
    a.append(i[1:])

one_hot_labels = np.asarray(a)

In [11]:
print(one_hot_labels.shape)

(150, 30)


In [12]:
X = X/255.0
X.shape

(150, 100, 100, 1)

In [13]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dropout(0.5))
model.add(Dense(128))


model.add(Dense(30))
model.add(Activation('sigmoid'))


#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 47, 47, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 64)        0         
__________

In [21]:
history = model.fit(X, one_hot_labels, batch_size=32, epochs=100, validation_split=0.3)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 12ms/step - loss: 0.1080 - acc: 0.9714 - val_loss: 0.2668 - val_acc: 0.9667
Epoch 2/100
105/105 [==============================] - 1s 11ms/step - loss: 0.0875 - acc: 0.9775 - val_loss: 0.2930 - val_acc: 0.9667
Epoch 3/100
105/105 [==============================] - 1s 12ms/step - loss: 0.0774 - acc: 0.9797 - val_loss: 0.3275 - val_acc: 0.9667
Epoch 4/100
105/105 [==============================] - 1s 11ms/step - loss: 0.0669 - acc: 0.9803 - val_loss: 0.3525 - val_acc: 0.9667
Epoch 5/100
105/105 [==============================] - 1s 11ms/step - loss: 0.0575 - acc: 0.9803 - val_loss: 0.3875 - val_acc: 0.9652
Epoch 6/100
105/105 [==============================] - 1s 13ms/step - loss: 0.0455 - acc: 0.9810 - val_loss: 0.4300 - val_acc: 0.9548
Epoch 7/100
105/105 [==============================] - 1s 12ms/step - loss: 0.0364 - acc: 0.9813 - val_loss: 0.4891 - val_acc: 0.9452
Epoch 8/100
105/1

In [22]:
def prepare(filepath):
    #IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) /255

In [26]:
prediction2 = model.predict([prepare('./sample_Signature/genuine/NFI-00105001.png')]) 

In [27]:
print(prediction2[0])
max(prediction2[0])

[1.0000000e+00 1.0667530e-18 9.4473496e-13 2.3434093e-14 5.6725839e-16
 9.0559340e-12 7.4242758e-11 2.6922689e-09 5.2660700e-17 2.0469978e-21
 2.7430686e-11 3.5864953e-12 4.9529733e-13 5.2434553e-11 5.0080252e-12
 2.1348641e-09 5.9637605e-16 6.2071028e-09 7.1153707e-15 4.9689214e-15
 2.0086646e-18 2.1193373e-13 8.3045311e-13 1.7862229e-18 1.4398442e-14
 1.6640746e-16 6.3685333e-20 1.9917835e-21 4.9510551e-17 1.1587745e-20]


1.0